In [198]:
import pandas as pd
from sklearn.linear_model import MultiTaskLasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from math import sqrt
import numpy as np
import seaborn as sns

CS (показатель цвета) :
представляет показатель цвета образца меда в диапазоне от 1,0 до 10,0. Более низкие значения указывают на более светлый цвет, а более высокие значения указывают на более темный цвет.

Плотность :
представляет плотность образца меда в граммах на кубический сантиметр при температуре 25°C в диапазоне от 1,21 до 1,86.

WC (Содержание воды) :
Показывает содержание воды в образце меда в диапазоне от 12,0% до 25,0%.

pH :
представляет уровень pH образца меда в диапазоне от 2,50 до 7,50.

EC (электрическая проводимость) :
представляет электропроводность образца меда в миллисименсах на сантиметр.

F (уровень фруктозы) :
представляет уровень фруктозы в образце меда в диапазоне от 20 до 50.

G (уровень глюкозы) :
представляет уровень глюкозы в образце меда в диапазоне от 20 до 45.

Анализ пыльцы :
представляет цветочный источник образца меда. Возможные значения: клевер, полевой цветок, апельсиновый цвет, люцерна, акация, лаванда, эвкалипт, гречиха, манука, шалфей, подсолнечник, бурачник, розмарин, тимьян, вереск, тупело, черника, каштан и авокадо.

Вязкость :
представляет вязкость образца меда в сантипуазах в диапазоне от 1500 до 10000. Значения вязкости от 2500 до 9500 считаются оптимальными для чистоты.

Чистота :
чистота образца меда в диапазоне от 0,01 до 1,00.

Цена :
Расчетная цена меда.

In [199]:
df = pd.read_csv("honey_purity_dataset.csv")
df.head(5)

,CS,Density,WC,pH,EC,F,G,Pollen_analysis,Viscosity,Purity,Price
0,2.81,1.75,23.04,6.29,0.76,39.02,33.63,Blueberry,4844.50,0.68,645.24
1,9.47,1.82,17.50,7.20,0.71,38.15,34.41,Alfalfa,6689.02,0.89,385.85
2,4.61,1.84,23.72,7.31,0.80,27.47,34.36,Chestnut,6883.60,0.66,639.64
3,1.77,1.40,16.61,4.01,0.78,31.52,28.15,Blueberry,7167.56,1.00,946.46
4,6.11,1.25,19.63,4.82,0.90,29.65,42.52,Alfalfa,5125.44,1.00,432.62


In [200]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247903 entries, 0 to 247902
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   CS               247903 non-null  float64
 1   Density          247903 non-null  float64
 2   WC               247903 non-null  float64
 3   pH               247903 non-null  float64
 4   EC               247903 non-null  float64
 5   F                247903 non-null  float64
 6   G                247903 non-null  float64
 7   Pollen_analysis  247903 non-null  object 
 8   Viscosity        247903 non-null  float64
 9   Purity           247903 non-null  float64
 10  Price            247903 non-null  float64
dtypes: float64(10), object(1)
memory usage: 20.8+ MB


In [201]:
df.describe()

,CS,Density,WC,pH,EC,F,G,Viscosity,Purity,Price
count,247903.000000,247903.000000,247903.000000,247903.000000,247903.000000,247903.000000,247903.000000,247903.000000,247903.000000,247903.000000
mean,5.500259,1.535523,18.502625,4.996047,0.799974,34.970573,32.501006,5752.893888,0.824471,594.807644
std,2.593947,0.187824,3.748635,1.444060,0.057911,8.655898,7.226290,2455.739903,0.139417,233.627972
min,1.000000,1.210000,12.000000,2.500000,0.700000,20.000000,20.000000,1500.050000,0.610000,128.720000
25%,3.260000,1.370000,15.260000,3.750000,0.750000,27.460000,26.230000,3627.880000,0.660000,433.000000
50%,5.500000,1.540000,18.510000,4.990000,0.800000,34.970000,32.490000,5753.770000,0.820000,612.960000
75%,7.740000,1.700000,21.750000,6.250000,0.850000,42.470000,38.760000,7886.650000,0.970000,770.220000
max,10.000000,1.860000,25.000000,7.500000,0.900000,50.000000,45.000000,9999.970000,1.000000,976.690000


In [202]:
df["Pollen_analysis"].unique()

array(['Blueberry', 'Alfalfa', 'Chestnut', 'Borage', 'Sunflower',
       'Orange Blossom', 'Acacia', 'Tupelo', 'Clover', 'Wildflower',
       'Thyme', 'Sage', 'Avocado', 'Lavender', 'Eucalyptus', 'Buckwheat',
       'Rosemary', 'Heather', 'Manuka'], dtype=object)

In [203]:
from category_encoders.target_encoder import TargetEncoder
scaler = TargetEncoder()
df['Pollen_analysis']=scaler.fit_transform(X=df['Pollen_analysis'],y=df['Price'])

In [204]:
df.corr()

,CS,Density,WC,pH,EC,F,G,Pollen_analysis,Viscosity,Purity,Price
CS,1.000000,-0.001099,0.001148,0.000267,-0.000215,-0.003287,0.001217,0.000385,-0.000099,0.079770,0.035166
Density,-0.001099,1.000000,-0.000037,0.002389,-0.004113,0.000515,0.002244,-0.004194,-0.003295,-0.108834,-0.050518
WC,0.001148,-0.000037,1.000000,0.001068,0.000690,0.000912,0.003517,-0.001623,-0.001088,-0.028894,-0.014381
pH,0.000267,0.002389,0.001068,1.000000,-0.001400,0.000511,-0.000725,-0.001886,-0.002347,-0.230855,-0.100714
EC,-0.000215,-0.004113,0.000690,-0.001400,1.000000,-0.001773,-0.002520,-0.001254,-0.001755,0.000151,-0.000571
F,-0.003287,0.000515,0.000912,0.000511,-0.001773,1.000000,-0.000683,-0.001668,-0.000023,-0.001149,-0.002041
G,0.001217,0.002244,0.003517,-0.000725,-0.002520,-0.000683,1.000000,-0.005660,0.000453,0.000059,-0.004815
Pollen_analysis,0.000385,-0.004194,-0.001623,-0.001886,-0.001254,-0.001668,-0.005660,1.000000,0.004499,0.000780,0.889155
Viscosity,-0.000099,-0.003295,-0.001088,-0.002347,-0.001755,-0.000023,0.000453,0.004499,1.000000,0.012572,0.009632
Purity,0.079770,-0.108834,-0.028894,-0.230855,0.000151,-0.001149,0.000059,0.000780,0.012572,1.000000,0.432581


## Регрессия

In [205]:
y = df[['Purity','Price']]
X =df.drop(['Purity','Price'], axis= 1)

In [206]:
y

,Purity,Price
0,0.68,645.24
1,0.89,385.85
2,0.66,639.64
3,1.00,946.46
4,1.00,432.62
...,...,...
247898,1.00,754.98
247899,1.00,543.41
247900,0.64,615.46
247901,0.97,949.32


In [207]:
X

,CS,Density,WC,pH,EC,F,G,Pollen_analysis,Viscosity
0,2.81,1.75,23.04,6.29,0.76,39.02,33.63,779.306635,4844.50
1,9.47,1.82,17.50,7.20,0.71,38.15,34.41,356.417632,6689.02
2,4.61,1.84,23.72,7.31,0.80,27.47,34.36,795.422929,6883.60
3,1.77,1.40,16.61,4.01,0.78,31.52,28.15,779.306635,7167.56
4,6.11,1.25,19.63,4.82,0.90,29.65,42.52,356.417632,5125.44
...,...,...,...,...,...,...,...,...,...
247898,1.98,1.29,17.90,4.82,0.89,36.10,34.69,622.343930,8261.63
247899,6.18,1.67,19.54,4.91,0.85,31.15,20.82,446.981373,6939.39
247900,7.78,1.49,15.78,5.69,0.73,44.60,44.07,795.422929,4139.79
247901,5.78,1.74,14.96,6.81,0.83,47.19,37.79,804.771376,4417.74


In [208]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [209]:
parameters = {'alpha': np.arange(0, 1, 0.1)}

lasso_optimal = GridSearchCV(MultiTaskLasso(), parameters).fit(X_train, y_train)
lasso_optimal.best_params_

C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:2418: UserWarning: Coordinate descent with l1_reg=0 may lead to unexpected results and is discouraged.
  ) = cd_fast.enet_coordinate_descent_multi_task(
C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:2418: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 849392178.54038, tolerance: 863999.692444925
  ) = cd_fast.enet_coordinate_descent_multi_task(
C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:2418: UserWarning: Coordinate descent with l1_reg=0 may lead to unexpected results and is discouraged.
  ) = cd_fast.enet_coordinate_descent_multi_task(
C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:2418: ConvergenceWarning: Objective did not converge. You might want to increase the number of iter

{'alpha': 0.0}

In [210]:
Input = [('minmaxscaler', MinMaxScaler()),('model', MultiTaskLasso(alpha = lasso_optimal.best_params_['alpha']))]

pipe = Pipeline(Input)
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'R^2: {pipe.score(X_test, y_test)}')

C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:2418: UserWarning: Coordinate descent with l1_reg=0 may lead to unexpected results and is discouraged.
  ) = cd_fast.enet_coordinate_descent_multi_task(


MAE: 43.17028019275427
MSE: 5334.0716329010575
RMSE: 73.03472895069207
MAPE: 0.15079982899868505
R^2: 0.43748054257352076


C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:2418: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1061751178.801345, tolerance: 1081117.804376188
  ) = cd_fast.enet_coordinate_descent_multi_task(


In [211]:
Input = [('model', MultiTaskLasso(alpha = lasso_optimal.best_params_['alpha']))]

pipe = Pipeline(Input)
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'R^2: {pipe.score(X_test, y_test)}')

C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:2418: UserWarning: Coordinate descent with l1_reg=0 may lead to unexpected results and is discouraged.
  ) = cd_fast.enet_coordinate_descent_multi_task(


MAE: 43.170280192754355
MSE: 5334.071632901036
RMSE: 73.03472895069191
MAPE: 0.15079982899868014
R^2: 0.4374805425735206


C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:2418: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1061751178.8013446, tolerance: 1081117.804376188
  ) = cd_fast.enet_coordinate_descent_multi_task(


## Классификация

как kmeans, но данные размечены

In [214]:
df = pd.read_csv("honey_purity_dataset.csv")
df['Pollen_analysis']=df['Pollen_analysis'].astype("category").cat.codes

y = df[['Pollen_analysis']]
X =df.drop(['Pollen_analysis'], axis= 1)

In [215]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)

In [216]:
centroid = NearestCentroid().fit(X_train, y_train)
y_pred = centroid.predict(X_test)

print(f"Training Set Score : {centroid.score(X_train, y_train) * 100} %")
print(f"Test Set Score : {centroid.score(X_test, y_test) * 100} %")

Training Set Score : 11.164167364185516 %
Test Set Score : 10.863032209919123 %


C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [217]:
Input = [('minmaxscaler', MinMaxScaler()), ('model', NearestCentroid()) ]

pipe = Pipeline(Input)
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [218]:
print(f"Training Set Score : {pipe.score(X_train, y_train) * 100} %")
print(f"Test Set Score : {pipe.score(X_test, y_test) * 100} %")

Training Set Score : 17.80790835106544 %
Test Set Score : 17.22837377221113 %
